In [12]:
esg_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/3am_all.csv', encoding='cp949')

In [13]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

  #제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

#####뉴스크롤링 시작#####

results_df= pd.DataFrame()
for corp_name in esg_df['기업명']:

    #검색어 입력
    search = corp_name
    #검색 시작할 페이지 입력
    page = 1
    page2 = 2

    # naver url 생성
    url = makeUrl(search,page,page2)

    #뉴스 크롤러 실행
    news_titles = []
    news_url =[]
    news_contents =[]
    news_dates = []
    for i in url:
        url = articles_crawler(url)
        news_url.append(url)


    #제목, 링크, 내용 담을 리스트 생성
    news_url_1 = []

    #1차원 리스트로 만들기(내용 제외)
    makeList(news_url_1,news_url)

    #NAVER 뉴스만 남기기
    final_urls = []
    for i in tqdm(range(len(news_url_1))):
        if "news.naver.com" in news_url_1[i]:
            final_urls.append(news_url_1[i])
        else:
            pass

# 뉴스 내용 크롤링

    for i in tqdm(final_urls):
        #각 기사 html get하기
        news = requests.get(i,headers=headers)
        news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
        content = news_html.select("article#dic_area")
        if content == []:
            content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
        content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        try:
            html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
        news_dates.append(news_date)

#데이터 프레임 만들기
    news_df = pd.DataFrame({'corp_name': corp_name, 'date':news_dates,'title':news_titles,'content':news_contents})

    #중복 행 지우기
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
    results_df = pd.concat([results_df, news_df], axis=0, ignore_index=True)

생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AJ네트웍스&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AJ네트웍스&start=11']


100%|██████████| 7/7 [00:05<00:00,  1.32it/s]


생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AK홀딩스&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=AK홀딩스&start=11']


100%|██████████| 5/5 [00:03<00:00,  1.43it/s]


생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=BGF&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=BGF&start=11']


...

100%|██████████| 10/10 [00:07<00:00,  1.30it/s]


생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=효성화학&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=효성화학&start=11']


100%|██████████| 12/12 [00:09<00:00,  1.30it/s]


생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=후성&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=후성&start=11']


100%|██████████| 6/6 [00:04<00:00,  1.36it/s]


In [14]:
results_df

,corp_name,date,title,content
0,AJ네트웍스,2024-07-24 16:54:10,"[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 마련",[\n이 기사는 2024년 07월 19일 15시 40분 넘버스에 발행된 기사입니다....
1,AJ네트웍스,2024-07-12 13:22:15,"한신평, AJ네트웍스 BBB+로 유지",[\n\t\t\t [파이낸셜뉴스] 한국신용평가는 AJ네트웍스의 신용등급을 BBB+...
2,AJ네트웍스,2024-07-17 18:10:07,"[마켓인]등급 하향 우려에도…풀무원, 신종자본증권 목표액 조달","[\n풀무원, 신종자본증권 700억 모집에 980억 주문“당분간 재무부담 완화 쉽지..."
3,AJ네트웍스,2024-07-04 10:33:10,"[이호 기자의 마켓ON]SK지오센트릭·대신에프앤아이·AJ네트웍스, 나란히 회사채 발행","[\n\t\t\tSK지오센트릭과 대신에프앤아이, AJ네트웍스가 나란히 회사채를 발행..."
4,AJ네트웍스,2024-06-28 09:51:18,"AJ네트웍스, 300억 규모 녹색채권 발행",[\n\n\n\n\nAJ네트웍스는 300억원 규모의 녹색채권을 발행했다고 28일 밝...
...,...,...,...,...
7242,후성,2024-08-06 16:01:09,"""술고래는 노화도 빠르다?""...술 많이 마실수록 더 늙어, 왜?",[\n알코올 치료 프로그램으로 개선할 수 있어\n\n\n\n알코올 의존증이 생물학적...
7243,후성,2024-08-05 16:26:14,'이 식단' 딱 두 달 했을 뿐인데… 신체 노화 느려졌다,[\n\n\n\n\n단기간만 채식 위주 식단을 실천해도 노화를 늦출 수 있다는 연구...
7244,후성,2024-07-31 11:30:08,생물학적 노화의 주범은 ‘설탕’…첨가당 1g만 먹어도 영향,[\n\n\n\n\n사진=게티이미지코리아.젊은 외모를 오랫동안 유지하고 싶다면 당장...
7245,후성,2024-08-01 07:31:11,"하루에 ‘이것’ 10g 덜 먹으면, 노화 2.4개월 늦춘다",[\n\n\n\n\n매일 설탕 섭취량을 10g씩만 줄여도 생물학적 나이를 낮출 수 ...


In [16]:
results_df.to_csv('/content/drive/MyDrive/Colab Notebooks/df_news.csv', index=False, encoding='utf-8-sig')

In [17]:
results_df

,corp_name,date,title,content
0,AJ네트웍스,2024-07-24 16:54:10,"[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 마련",[\n이 기사는 2024년 07월 19일 15시 40분 넘버스에 발행된 기사입니다....
1,AJ네트웍스,2024-07-12 13:22:15,"한신평, AJ네트웍스 BBB+로 유지",[\n\t\t\t [파이낸셜뉴스] 한국신용평가는 AJ네트웍스의 신용등급을 BBB+...
2,AJ네트웍스,2024-07-17 18:10:07,"[마켓인]등급 하향 우려에도…풀무원, 신종자본증권 목표액 조달","[\n풀무원, 신종자본증권 700억 모집에 980억 주문“당분간 재무부담 완화 쉽지..."
3,AJ네트웍스,2024-07-04 10:33:10,"[이호 기자의 마켓ON]SK지오센트릭·대신에프앤아이·AJ네트웍스, 나란히 회사채 발행","[\n\t\t\tSK지오센트릭과 대신에프앤아이, AJ네트웍스가 나란히 회사채를 발행..."
4,AJ네트웍스,2024-06-28 09:51:18,"AJ네트웍스, 300억 규모 녹색채권 발행",[\n\n\n\n\nAJ네트웍스는 300억원 규모의 녹색채권을 발행했다고 28일 밝...
...,...,...,...,...
7242,후성,2024-08-06 16:01:09,"""술고래는 노화도 빠르다?""...술 많이 마실수록 더 늙어, 왜?",[\n알코올 치료 프로그램으로 개선할 수 있어\n\n\n\n알코올 의존증이 생물학적...
7243,후성,2024-08-05 16:26:14,'이 식단' 딱 두 달 했을 뿐인데… 신체 노화 느려졌다,[\n\n\n\n\n단기간만 채식 위주 식단을 실천해도 노화를 늦출 수 있다는 연구...
7244,후성,2024-07-31 11:30:08,생물학적 노화의 주범은 ‘설탕’…첨가당 1g만 먹어도 영향,[\n\n\n\n\n사진=게티이미지코리아.젊은 외모를 오랫동안 유지하고 싶다면 당장...
7245,후성,2024-08-01 07:31:11,"하루에 ‘이것’ 10g 덜 먹으면, 노화 2.4개월 늦춘다",[\n\n\n\n\n매일 설탕 섭취량을 10g씩만 줄여도 생물학적 나이를 낮출 수 ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
esgdf = pd.DataFrame({
    '기업명':['t']})

In [4]:
import pandas as pd
import requests

# 예시 esg_df DataFrame (기업명 컬럼 포함)
esg_df = pd.DataFrame({'기업명': ['삼성전자', '하이닉스'] })

# 최종 결과를 저장할 DataFrame 초기화
results_df = pd.DataFrame()

# esg_df의 기업명 컬럼에 있는 각 기업명을 반복
for company_name in esg_df['기업명']:
    for page in range(1, 4):  # 1페이지부터 3페이지까지
        # search 값 설정
        search = company_name

        # 검색할 URL 및 파라미터 설정 (예시 URL 및 파라미터)
        url = 'https://example.com/search'  # 실제 API URL로 교체
        params = {
            'query': search,
            'page': page
        }

        # API 호출
        response = requests.get(url, params=params)

        if response.status_code == 200:
            # 검색 결과를 DataFrame으로 변환
            search_results = pd.DataFrame(response.json()['list'])

            # 결과에 기업명을 추가
            search_results['기업명'] = company_name

            # results_df에 결과를 합치기
            results_df = pd.concat([results_df, search_results], axis=0, ignore_index=True)
        else:
            print(f"Failed to retrieve data for {company_name} on page {page}")

# 최종 결과 확인
print(results_df)


Failed to retrieve data for 삼성전자 on page 1
Failed to retrieve data for 삼성전자 on page 2
Failed to retrieve data for 삼성전자 on page 3
Failed to retrieve data for 하이닉스 on page 1
Failed to retrieve data for 하이닉스 on page 2
Failed to retrieve data for 하이닉스 on page 3
Empty DataFrame
Columns: []
Index: []
